In [ ]:
# NOTE : The code here doesn't work but I get it

"""
Transfer learning is a machine learning method where a model developed for a first task is then reused
for a model on a second task
For example, we can train a model to classify birds and cats, then use the same model, modified only
in the last layer to classify bees and dogs
This allows for the rapid generation of new models and this is super important as training a new model
can be very time consuming
Tranfer learning can acieve pretty good results
We just want to replace the classification layer (flatter, fully connected, softmax)
"""

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
"""
In this example, we are using the pre-trained resnet18 CNN, trained on more than a million images
from the imagenet database
It is 18 layers deep and can classify images into 1000 object categories
We have only 2 classes in our example - we only want to detect bees and ants
"""

In [3]:
# Setup device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [5]:
# Import data

data_dir = 'data/gymenoptera_data'
sets = ['train', 'val']

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                 for x in ['train', 'val']}

downloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                             batch_size=4,
                                             shuffle = True,)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(clas_names)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/gymenoptera_data\\train'

In [ ]:
model = models.resnet18(pretrained = True) # Already optimized weights on imagenet data


# We want to exchange the last fully-trained layer

# First, get number of input features from last layer
num_features = model.fc.in_features # fc = fully connected layer

# Create new layer, assign it to last layer

model.fc = nn.Linear(num_features, 2) # Give it a new fully connected layer
# Only 2 outputs now (bees and ants)

model.to(device) # To GPU if possible

# Define loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters, lr=0.001)

# NEW - Scheduler
# Update learning rate

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # In torch.optimizer module
# Every 7 epochs, our lr is multiplied by gamma
"""
for epoch in range(100):
    train() # optmizer.step
    evaluate() 
    scheduler.step() # This is how we use a shceduler
"""

# Call training functions here

In [ ]:
"""
In this case, we are using "fine tuning", meainig that we train the whole model again but only a 
little bit (fine tuning the weights based on the new data) with a new last layer
"""

In [ ]:
# As a second option, we can freeze all layers in beginning and only train lasy layer

model = models.resnet18(pretrained = True) # Already optimized weights on imagenet data

for param in model.parameters():
    param.requires_grad = False # FREEZES ALL LAYERS IN BEGINNING
    # SO WE ARE ONLY TRAINING LAST LAYER

# We want to exchange the last fully-trained layer

# First, get number of input features from last layer
num_features = model.fc.in_features # fc = fully connected layer

# Create new layer, assign it to last layer

model.fc = nn.Linear(num_features, 2) # Give it a new fully connected layer
# Only 2 outputs now (bees and ants)

model.to(device) # To GPU if possible

# Define loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters, lr=0.001)

# NEW - Scheduler
# Update learning rate

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # In torch.optimizer module

 
# Train data here 